# Preparación del entorno

#### Se clona el repositorio original

In [185]:
!rm -rf /content/*
!git clone https://github.com/facebookresearch/SymbolicMathematics
!mv -f SymbolicMathematics/* .
!rm -rf SymbolicMathematics

Cloning into 'SymbolicMathematics'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 28 (delta 2), reused 0 (delta 0), pack-reused 20
Unpacking objects: 100% (28/28), done.


In [186]:
!ls

beam_integration.ipynb	CONTRIBUTING.md  main.py    split_data.py
CODE_OF_CONDUCT.md	LICENSE		 README.md  src


#### Se importan las bibliotecas necesarias

In [187]:
import os
import numpy as np
import sympy as sp
import torch

from src.utils import AttrDict
from src.envs import build_env
from src.model import build_modules

from src.utils import to_cuda
from src.envs.sympy_utils import simplify

#### Se descarga el modelo pre-entrenado

In [188]:
!rm -f fwd_bwd_ibp.pth
#!wget https://dl.fbaipublicfiles.com/SymbolicMathematics/models/fwd_bwd_ibp.pth
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1plWyWdmpmx4IQ-oU0Ykwu0h1rVcaPBFm' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1plWyWdmpmx4IQ-oU0Ykwu0h1rVcaPBFm" -O fwd_bwd_ibp.pth && rm -rf /tmp/cookies.txt

--2022-12-04 22:07:14--  https://docs.google.com/uc?export=download&confirm=t&id=1plWyWdmpmx4IQ-oU0Ykwu0h1rVcaPBFm
Resolving docs.google.com (docs.google.com)... 74.125.24.102, 74.125.24.101, 74.125.24.113, ...
Connecting to docs.google.com (docs.google.com)|74.125.24.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-14-bk-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/9iu9rh1p8ltlvmgm229krhhk5njsp0bb/1670191575000/01049389482803376309/*/1plWyWdmpmx4IQ-oU0Ykwu0h1rVcaPBFm?e=download&uuid=738b08f8-a2db-4e76-bbe6-4074e01ba92c [following]
--2022-12-04 22:07:15--  https://doc-14-bk-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/9iu9rh1p8ltlvmgm229krhhk5njsp0bb/1670191575000/01049389482803376309/*/1plWyWdmpmx4IQ-oU0Ykwu0h1rVcaPBFm?e=download&uuid=738b08f8-a2db-4e76-bbe6-4074e01ba92c
Resolving doc-14-bk-docs.googleusercontent.com (doc-14-bk-docs.googleusercontent.com)... 142.251.12.132, 2404

In [189]:
model_path = './fwd_bwd_ibp.pth'
assert os.path.isfile(model_path)

#### Se definen los parámetros del modelo

In [190]:
params = params = AttrDict({

    # environment parameters
    'env_name': 'char_sp',
    'int_base': 10,
    'balanced': False,
    'positive': True,
    'precision': 10,
    'n_variables': 1,
    'n_coefficients': 0,
    'leaf_probs': '0.75,0,0.25,0',
    'max_len': 512,
    'max_int': 5,
    'max_ops': 15,
    'max_ops_G': 15,
    'clean_prefix_expr': True,
    'rewrite_functions': '',
    'tasks': 'prim_fwd',
    'operators': 'add:10,sub:3,mul:10,div:5,sqrt:4,pow2:4,pow3:2,pow4:1,pow5:1,ln:4,exp:4,sin:4,cos:4,tan:4,asin:1,acos:1,atan:1,sinh:1,cosh:1,tanh:1,asinh:1,acosh:1,atanh:1',

    # model parameters
    'cpu': False,
    'emb_dim': 1024,
    'n_enc_layers': 6,
    'n_dec_layers': 6,
    'n_heads': 8,
    'dropout': 0,
    'attention_dropout': 0,
    'sinusoidal_embeddings': False,
    'share_inout_emb': True,
    'reload_model': model_path,

})

#### Se construyen los módulos codificador y decodificador

In [191]:
env = build_env(params)
x = env.local_dict['x']

In [192]:
modules = build_modules(env, params)
encoder = modules['encoder']
decoder = modules['decoder']

# Inicio de la demostración

El modelo posee las siguientes características:

* Cabezas de atención: 8
* Número de capas: 6
* Tamaño del embedding: 1024
* Número de parámetros (encoder): 79,866,880
* Número de parámetros (decoder): 10,5069,659

<center><img src="http://www.iic.uam.es/wp-content/uploads/2020/05/arquitectura-transformers.jpg" alt="drawing" width="250"/></center>

#### Se construye una función F, se calcula su derivada f = F', y se calcula la integral de f para obtener F.

In [193]:
# Ejemplos de fórmulas
F_infix = 'x * tan(exp(x)/x)'
F_infix = 'x * cos(x**2) * tan(x)'
F_infix = 'cos(x**2 * exp(x * cos(x)))'
F_infix = 'ln(cos(x + exp(x)) * sin(x**2 + 2) * exp(x) / x)'
F_infix = '-10*x**2'

In [194]:
# F es lo que el modelo intentará calcular
F = sp.S(F_infix, locals=env.local_dict)
F

-10*x**2

In [195]:
# f es la entrada del modelo, es la derivada de F.
f = F.diff(x)
f

-20*x

#### Convierte los objetos de Sympy a una lista de tokens

In [196]:
F_prefix = env.sympy_to_prefix(F)
f_prefix = env.sympy_to_prefix(f)
print(f"F prefix: {F_prefix}")
print(f"f prefix: {f_prefix}")

F prefix: ['mul', 'INT-', '1', '0', 'pow', 'x', 'INT+', '2']
f prefix: ['mul', 'INT-', '2', '0', 'x']


### Encode de la entrada

La entrada se asume que está igualada a 0. Por lo que la expresión de entrada se define como $Y' - f$

In [197]:
x1_prefix = env.clean_prefix(['sub', 'derivative', 'f', 'x', 'x'] + f_prefix)
print(x1_prefix)

['sub', "Y'", 'mul', 'INT-', '2', '0', 'x']


Posteriormente se agrega un token al inicio y al final de la ecuación y cada token se convierte a su correspondiente índice del vocabulario. La función word2id toma cada token y lo mapea a un número entero del tamaño del vocabulario (0-90).

In [198]:
x1 = torch.LongTensor(
    [env.eos_index] +
    [env.word2id[w] for w in x1_prefix] +
    [env.eos_index]
).view(-1, 1)
print(x1)

tensor([[ 0],
        [67],
        [79],
        [54],
        [72],
        [83],
        [81],
        [12],
        [ 0]])


El vocabulario se compone de la siguiente manera:

words: {'\<s\>': 0, '\</s\>': 1, '\<pad\>': 2, '(': 3, ')': 4, '\<SPECIAL_5\>': 5, '\<SPECIAL_6\>': 6, '\<SPECIAL_7\>': 7,
 '\<SPECIAL_8\>': 8, '\<SPECIAL_9\>': 9, 'pi': 10, 'E': 11, 'x': 12, 'y': 13, 'z': 14, 't': 15, 'a0': 16, 'a1': 17, 'a2': 18,
 'a3': 19, 'a4': 20, 'a5': 21, 'a6': 22, 'a7': 23, 'a8': 24, 'a9': 25, 'abs': 26, 'acos': 27, 'acosh': 28, 'acot': 29,
 'acoth': 30, 'acsc': 31, 'acsch': 32, 'add': 33, 'asec': 34, 'asech': 35, 'asin': 36, 'asinh': 37, 'atan': 38,
 'atanh': 39, 'cos': 40, 'cosh': 41, 'cot': 42, 'coth': 43, 'csc': 44, 'csch': 45, 'derivative': 46, 'div': 47,
 'exp': 48, 'f': 49, 'g': 50, 'h': 51, 'inv': 52, 'ln': 53, 'mul': 54, 'pow': 55, 'pow2': 56, 'pow3': 57,
 'pow4': 58, 'pow5': 59, 'rac': 60, 'sec': 61, 'sech': 62, 'sign': 63, 'sin': 64, 'sinh': 65, 'sqrt': 66,
 'sub': 67, 'tan': 68, 'tanh': 69, 'I': 70, 'INT+': 71, 'INT-': 72, 'INT': 73, 'FLOAT': 74, '-': 75,
 '.': 76, '10^': 77, 'Y': 78, "Y'": 79, "Y''": 80, '0': 81, '1': 82, '2': 83, '3': 84, '4': 85, '5': 86,
 '6': 87, '7': 88, '8': 89, '9': 90}

#### Se ejecuta el encoder con la entrada x1

In [199]:
len1 = torch.LongTensor([len(x1)])
x1, len1 = to_cuda(x1, len1)

with torch.no_grad():
    encoded = encoder('fwd', x=x1, lengths=len1, causal=False).transpose(0, 1)

print(encoded.shape)
print(encoded)

torch.Size([1, 9, 1024])
tensor([[[ 0.0082, -0.0093,  0.0040,  ...,  0.0202, -0.0030,  0.0033],
         [ 0.0302, -0.0190, -0.0334,  ..., -0.0268, -0.0020, -0.0508],
         [-0.0253,  0.0015,  0.0135,  ..., -0.0758, -0.0060, -0.0510],
         ...,
         [ 0.0131,  0.0359, -0.0006,  ...,  0.1034, -0.0196,  0.0115],
         [-0.0045, -0.0475, -0.0237,  ...,  0.0602,  0.0118,  0.0252],
         [ 0.0082, -0.0093,  0.0041,  ...,  0.0202, -0.0030,  0.0033]]],
       device='cuda:0')


### Decodifica con beam search

In [200]:
beam_size = 15
with torch.no_grad():
    _, _, beam = decoder.generate_beam(encoded, len1, beam_size=beam_size, length_penalty=1.0, early_stopping=1, max_len=200)
    assert len(beam) == 1
hypotheses = beam[0].hyp
assert len(hypotheses) == beam_size

/content/src/model/transformer.py:513: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = idx // n_words


El decodificador arroja las hipotesis como un vector de números asociados a los tokens del vocabulario.

In [201]:
print("Número de hipótesis:", len(hypotheses))
print("Primer hipótesis obtenida del decodificador:", hypotheses[0])

Número de hipótesis: 15
Primer hipótesis obtenida del decodificador: (-0.21639686160617405, tensor([ 0, 54, 72, 82, 81, 55, 12, 71, 83]))


### Mostrar las hipótesis y verificar las soluciones

In [210]:
print(f"Función de entrada (derivada) f: {f}")
print(f"Función original F: {F}", end="\n\n")

for score, sent in sorted(hypotheses, key=lambda x: x[0], reverse=True):

    # parse decoded hypothesis
    ids = sent[1:].tolist()                  # decoded token IDs
    tok = [env.id2word[wid] for wid in ids]  # convert to prefix

    try:
        hyp = env.prefix_to_infix(tok)       # convert to infix
        hyp = env.infix_to_sympy(hyp)        # convert to SymPy

        # check whether we recover f if we differentiate the hypothesis
        # note that sometimes, SymPy fails to show that hyp' - f == 0, and the result is considered as invalid, although it may be correct
        res = "Respuesta correcta" if simplify(hyp.diff(x) - f, seconds=1) == 0 else "Respuesta incorrecta"
        simpl = simplify(hyp, seconds=1)
        simpldiff = simplify(hyp.diff(x), seconds=1)

    except:
        res = "Expresión prefija inválida"
        hyp = tok

    print("%s  Beam socre: %.5f  Hipótesis: %s    \t Hipótesis simpl. %s   \t Hip. Diff. simpl. %s" % (res, score, hyp, simpl,simpldiff))

Función de entrada (derivada) f: -20*x
Función original F: -10*x**2

Respuesta correcta  Beam socre: -0.20796  Hipótesis: 2*x*(-5*x + 5/x)    	 Hipótesis simpl. 10 - 10*x**2   	 Hip. Diff. simpl. -20*x
Respuesta correcta  Beam socre: -0.21469  Hipótesis: -5*x*(2*x + 1) + 5*x    	 Hipótesis simpl. -10*x**2   	 Hip. Diff. simpl. -20*x
Respuesta correcta  Beam socre: -0.21640  Hipótesis: -10*x**2    	 Hipótesis simpl. -10*x**2   	 Hip. Diff. simpl. -20*x
Respuesta correcta  Beam socre: -0.22361  Hipótesis: -x*(10*x + 1) + x    	 Hipótesis simpl. -10*x**2   	 Hip. Diff. simpl. -20*x
Respuesta correcta  Beam socre: -0.23346  Hipótesis: (-10*x**3 + x)/x    	 Hipótesis simpl. 1 - 10*x**2   	 Hip. Diff. simpl. -20*x
Respuesta correcta  Beam socre: -0.25223  Hipótesis: 2*(-5*x**3 + x)/x    	 Hipótesis simpl. 2 - 10*x**2   	 Hip. Diff. simpl. -20*x
Respuesta correcta  Beam socre: -0.25685  Hipótesis: 5*(-2*x**3 + x)/x    	 Hipótesis simpl. 5 - 10*x**2   	 Hip. Diff. simpl. -20*x
Respuesta correc